In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests
import time
import datetime

In [41]:
session = requests.Session()

req = session.get('https://cooking.nytimes.com/recipes/1013062-cuban-black-beans?action=click&region=Sam%20Sifton%27s%20Suggestions&rank=1')
soup = BeautifulSoup(req.text, 'lxml')

In [42]:
#find title
soup.find('div', class_= 'title-container').get_text().strip()

'Cuban Black Beans'

In [43]:
#find time
soup.find('ul', class_= 'recipe-time-yield').find_all('li')[1].get_text().strip().split('\n')

['Time', '45 minutes']

In [51]:
#find star rating
int(soup.find('span', itemprop= "ratingValue").text)

5

In [66]:
#list of ingredients
items = []
for item in soup.find('ul', class_= 'recipe-ingredients').find_all('span', class_='ingredient-name'):
    items.append(item.text.strip())

items


['green peppers, stemmed and seeded',
 'garlic cloves',
 'pound dried black beans, rinsed and picked over to remove any stones',
 'smoked ham hock',
 'bay leaves',
 'teaspoons salt, or to taste',
 'cup olive oil',
 'slices thick bacon, cut into 1/2-inch pieces',
 'Spanish onion, diced',
 'jalapeño, stemmed and finely chopped',
 'teaspoon dried oregano',
 'teaspoon ground cumin',
 'teaspoon freshly ground black pepper',
 'tablespoons distilled white vinegar',
 'tablespoon turbinado or other brown sugar']

In [68]:
counts = []
for count in soup.find('ul', class_= 'recipe-ingredients').find_all('span', class_='quantity'):
    counts.append(count.text.strip())
    
counts

['1 ½', '10', '1', '1', '2', '5', '¼', '4', '1', '1', '1', '½', '½', '3', '1']

In [71]:
combined = []
for n in range(0,len(items)):
    combined.append(counts[n]+' '+items[n])
combined

['1 ½ green peppers, stemmed and seeded',
 '10 garlic cloves',
 '1 pound dried black beans, rinsed and picked over to remove any stones',
 '1 smoked ham hock',
 '2 bay leaves',
 '5 teaspoons salt, or to taste',
 '¼ cup olive oil',
 '4 slices thick bacon, cut into 1/2-inch pieces',
 '1 Spanish onion, diced',
 '1 jalapeño, stemmed and finely chopped',
 '1 teaspoon dried oregano',
 '½ teaspoon ground cumin',
 '½ teaspoon freshly ground black pepper',
 '3 tablespoons distilled white vinegar',
 '1 tablespoon turbinado or other brown sugar']

In [84]:
#cooking steps
steps = []

for step in soup.find('ol', class_= 'recipe-steps').find_all('li'):
    steps.append(step.text)

steps

['Cut 1 green pepper into 1-inch squares. Smash and peel 4 of the garlic cloves. Put the green pepper and garlic into a large pot with the beans, ham hock, bay leaves and 1 tablespoon salt. Add 2 quarts water and bring to a boil. Cover the pot and simmer until the beans are tender, an hour or more.',
 'Meanwhile, make a sofrito. Cut the remaining ½ green pepper into ¼-inch dice. Peel and finely chop the remaining garlic. Heat the olive oil in a very large skillet over medium-high heat. Add the bacon and cook, stirring occasionally, until it starts to brown, about 5 minutes. Add the green pepper and onion and cook, stirring, until slightly softened, about 3 minutes. Add the garlic, jalapeño (leave out the seeds if you don’t want it too spicy), oregano, cumin, black pepper and 2 teaspoons salt and stir for another minute. Pour in the vinegar and scrape any browned bits from bottom of pan with a wooden spoon. This is your sofrito.',
 'When the beans are cooked, discard the bay leaf. Remov

In [85]:
def recipe_compiler(url):
    req = session.get(url)
    soup = BeautifulSoup(req.text, 'lxml')
    
    title = soup.find('div', class_= 'title-container').get_text().strip()
    time = soup.find('ul', class_= 'recipe-time-yield').find_all('li')[1].get_text().strip().split('\n')[1]
    
    items = []
    for group in soup.find_all('ul', class_= 'recipe-ingredients'):
        for item in group.find_all('span', class_='ingredient-name'):
            items.append(item.text.strip())
            
    counts = []
    for group in soup.find_all('ul', class_= 'recipe-ingredients'):
        for item in group.find_all('span', class_='quantity'):
            counts.append(item.text.strip())
    
    combined = []
    for n in range(0,len(items)):
        combined.append(counts[n]+' '+items[n])
    
    steps = []

    for step in soup.find('ol', class_= 'recipe-steps').find_all('li'):
        steps.append(step.text)
    
    return {'recipe':title, 'cooking_time':time, 'item_list':combined,'instructions':steps}

In [86]:
url = 'https://cooking.nytimes.com/recipes/1013062-cuban-black-beans?action=click&region=Sam%20Sifton%27s%20Suggestions&rank=1'
recipe_compiler(url)

{'recipe': 'Cuban Black Beans',
 'cooking_time': '45 minutes',
 'item_list': ['1 ½ green peppers, stemmed and seeded',
  '10 garlic cloves',
  '1 pound dried black beans, rinsed and picked over to remove any stones',
  '1 smoked ham hock',
  '2 bay leaves',
  '5 teaspoons salt, or to taste',
  '¼ cup olive oil',
  '4 slices thick bacon, cut into 1/2-inch pieces',
  '1 Spanish onion, diced',
  '1 jalapeño, stemmed and finely chopped',
  '1 teaspoon dried oregano',
  '½ teaspoon ground cumin',
  '½ teaspoon freshly ground black pepper',
  '3 tablespoons distilled white vinegar',
  '1 tablespoon turbinado or other brown sugar'],
 'instructions': ['Cut 1 green pepper into 1-inch squares. Smash and peel 4 of the garlic cloves. Put the green pepper and garlic into a large pot with the beans, ham hock, bay leaves and 1 tablespoon salt. Add 2 quarts water and bring to a boil. Cover the pot and simmer until the beans are tender, an hour or more.',
  'Meanwhile, make a sofrito. Cut the remaining

In [89]:
url = 'https://cooking.nytimes.com/recipes/1020851-bittersweet-brownie-shortbread?action=click&module=Global%20Search%20Recipe%20Card&pgType=search&rank=74'
pd.DataFrame([recipe_compiler(url)])

,cooking_time,instructions,item_list,recipe
0,"1 1/2 hours, plus cooling",[Heat oven to 350 degrees. Grease a 9-by-13-in...,[1 ½ cups/340 grams cold unsalted butter (3 st...,Bittersweet Brownie Shortbread
